In [1]:
!pip install langchain chromadb transformers pytesseract torch torchvision



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain_core.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import pytesseract
from PIL import Image


In [2]:
from huggingface_hub import login
# Login to Hugging Face Hub
login()

In [3]:
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")

llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)
hf_llm = HuggingFacePipeline(pipeline=llm_pipeline)

Device set to use cuda:0
C:\Users\krish\AppData\Local\Temp\ipykernel_26428\3251464668.py:6: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf_llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [4]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma(persist_directory="database", embedding_function=embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

C:\Users\krish\AppData\Local\Temp\ipykernel_26428\459270609.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


C:\Users\krish\AppData\Local\Temp\ipykernel_26428\459270609.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="database", embedding_function=embedding_model)


In [5]:
prompt = PromptTemplate(
    input_variables=["context", "query"],
    template="""
You are a helpful assistant with deep chess knowledge. 
Use the following context to answer the user query:

Context:
{context}

Question: {query}

Answer:
""".strip()
)
llm_chain = LLMChain(llm=hf_llm, prompt=prompt)

C:\Users\krish\AppData\Local\Temp\ipykernel_26428\247015825.py:15: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=hf_llm, prompt=prompt)


In [6]:
from langchain_core.documents import Document

def rag_qa(query: str) -> str:
    docs: list[Document] = retriever.get_relevant_documents(query)
    context = "\n\n".join([doc.page_content for doc in docs])
    return llm_chain.invoke({"context": context, "query": query})["text"]

In [7]:
asr = pipeline("automatic-speech-recognition", model="openai/whisper-large-v2")
def transcribe_audio(path): return asr(path).get("text","")
def extract_text_from_image(path): return pytesseract.image_to_string(Image.open(path))

Device set to use cuda:0


In [8]:
def multimodal_agent(input_type: str, data_path: str) -> str:
    if input_type == "text":
        query = data_path
    elif input_type == "voice":
        query = transcribe_audio(data_path)
    elif input_type == "image":
        query = extract_text_from_image(data_path)
    else:
        return "Use 'text', 'voice', or 'image'."

    return rag_qa(query)

In [9]:
print(multimodal_agent("text", "what is checkmate?"))


C:\Users\krish\AppData\Local\Temp\ipykernel_26428\2604618252.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs: list[Document] = retriever.get_relevant_documents(query)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


You are a helpful assistant with deep chess knowledge. 
Use the following context to answer the user query:

Context:


Question: what is checkmate?

Answer: Checkmate is a chess term for a situation in which a player checksmate, or checks himself, by capturing an opponent's king.

Checkmate can occur in a game of chess or in other chess-like games, such as backgammon and draughts. Checkmate can also occur in a game of chess where there is no checkmate, but the game is still considered to be a draw. The most common method of achieving checkmate is to capture the opponent's king, but there are other methods. For example, in some games of chess, it is possible to checkmate by forcing the opponent to resign.

Checkmate is a crucial concept in chess, as it is a way of winning a game. It is often used as a way to end a game quickly, or to force the opponent to resign. It is also a way of demonstrating a player's mastery of the game.

Checkmate is a concept that is important to understand in

In [10]:
print(multimodal_agent("image", "C:\PES\CSSEM-6\GenAI\Project\CHESSBOT\SICILIANDEFENSE.png"))


<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\krish\AppData\Local\Temp\ipykernel_26428\1060204356.py:1: SyntaxWarning: invalid escape sequence '\P'
  print(multimodal_agent("image", "C:\PES\CSSEM-6\GenAI\Project\CHESSBOT\SICILIANDEFENSE.png"))
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


You are a helpful assistant with deep chess knowledge. 
Use the following context to answer the user query:

Context:


Question: WHAT IS SICILIAN DEFENSE?


Answer: SICILIAN DEFENSE is a chess opening where white plays a pawn to the fourth rank. The main idea is to attack the black king. 
In this opening, white's pieces are well-placed, and the black king is often caught in a dangerous position. It is a very powerful opening for white.


In [11]:
print(multimodal_agent("voice", "C:\PES\CSSEM-6\GenAI\Project\CHESSBOT\KINGSGAMBIT.mp3"))

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\krish\AppData\Local\Temp\ipykernel_26428\584048481.py:1: SyntaxWarning: invalid escape sequence '\P'
  print(multimodal_agent("voice", "C:\PES\CSSEM-6\GenAI\Project\CHESSBOT\KINGSGAMBIT.mp3"))
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\whisper\generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers 

You are a helpful assistant with deep chess knowledge. 
Use the following context to answer the user query:

Context:


Question:  What is King's Gambit?

Answer:  King's Gambit is a chess opening in which white plays 1.e4 e5 2.f4. White can then either continue with 3.d4, or play 3.d5.

Explanation: 

The King's Gambit is a chess opening in which white plays 1.e4 e5 2.f4. White can then either continue with 3.d4, or play 3.d5. The King's Gambit is named after the 17th-century English chess player and writer John King, who was the first to play it in a tournament in 1710. The opening is also known as the Sicilian Defense, and the 3.d5 line as the Nimzo-Indian Defense.

The King's Gambit is a gambit opening in which white plays 1.e4 e5 2.f4. White can then either continue with 3.d4, or play 3.d5. The King's Gambit is named after the 17th-century English chess player and writer John King, who was the first to play it in a tournament in 1710. The opening is also known as the Sicilian Defe